# Deterministic selection

(c) 2019 Manuel Razo. This work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/). All code contained herein is licensed under an [MIT license](https://opensource.org/licenses/MIT)

---

In [1]:
# Import our numerical workhorse
import numpy as np

# Our main plotting package (must have explicit import of submodules)
import bokeh.io
import bokeh.plotting
import bokeh.layouts
from bokeh.themes import Theme
from bokeh.models import CustomJS, WidgetBox, ColumnDataSource
from bokeh.models.widgets import Slider, RadioButtonGroup

# Enable viewing Bokeh plots in the notebook
bokeh.io.output_notebook()

Loading BokehJS ...

The first thing we need to define for the plot are the sliders that we will be able to manipulate. We will need two:
1. $s$ for the selection coefficient
2. $x_o$ for the initial frequency of the mutant.

In [38]:
# Define the interactions
s_select = Slider(title='s', start=-0.03, end=0.03, step=.0001, value=0)
x_init = Slider(title='x\u2092', start=0, end=1, step=0.05, value=0.25)

# Sanity check
bokeh.io.show(bokeh.layouts.column(s_select, x_init))

Now we are ready to prepare the data for the plot. Fro the very simple plot we want to generate all the data we need is a range of $t$ time values.
We will generate a `data source` for `bokeh`. This is the equivalent to a `pandas` dataframe where we could put different sources of data. This `data source` will include a precomputed allele frequency such that when the plot is first rendered it is not empty. So we will define a `python` function to compute this initial allele frequency. In the next step we will re-define the function in `javascript` such that the plot can be updated without the need of a `python` kernel.

In [39]:
# Define time array
time = np.linspace(0, 10000, 1000)

# Define function to compute allele frequency
def x_select(t, x_init, s_select):
    '''
    Computes the allele frequency x for the regime of deterministic
    selection acting on a one-locus two-allele system
    '''
    return x_init * np.exp(s_select * t) / \
    (1 + x_init * (np.exp(s_select * t) - 1))

# Define initial values of the parameters
x_i = 0.1
s_i = 0.1

# Generate bokeh data source
source = ColumnDataSource({'time': time,
                           'x_allele': x_select(time, x_i, s_i)})

/Users/razo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


Now we need to define the `JavaScript` callback function. We will define it using `JavaScript` such that the function can exists as its own standing HTML object without the need for a `python` kernel to render it. Since our function is very simple we can compute this very easily

In [40]:
# Define JavaScript callback function
cb_script = """
// Variable definition
var data = source.data; 
var s_select = sSlider.value;
var x_init = xoSlider.value;


// Function definition
function x_select(t, x_init, s_select){
    var updated_val =  x_init * Math.exp(s_select * t) / 
    (1 + x_init * (Math.exp(s_select * t) - 1));
    return updated_val;
}

// Update values using a for loop (since I don't know how to do 
// vectorized oprations)
var updated_allele = []; // temporary variable to update values
for (var i = 0; i < data['time'].length; i++){
    updated_allele[i] = x_select(data['time'][i], x_init, s_select);
}
// update x_allele column entry in source
data['x_allele'] = updated_allele

// Emit data source for plot to be updated
source.change.emit();
"""

Done! Not too bad. Now let's define the arguments for the callback function

In [41]:
# Define arguments for JavaScript callback function
cb_args = {'source': source, 'sSlider': s_select, 'xoSlider': x_init}
# Asign arguments to function
cb = CustomJS(args=cb_args, code=cb_script)

Now we must assign this callback function to each of the sliders. What this means is that we must indicate that every time the slider value is changed, the `JavaScript` callback function must be executed.

In [42]:
# Assign callback function to widgets
x_init.callback = cb
s_select.callback = cb
x_init.js_on_change('value', cb)
s_select.js_on_change('value', cb)

Alright. Now everything is setup for our interactive plot! Now we just need to define the bokeh plot.

In [43]:
# Define bokeh axis
x_allele_ax = bokeh.plotting.figure(width=300, height=275,
                                    x_axis_label='time (a.u.)',
                                    y_axis_label='allele frequency',
                                    y_range=[0, 1.05])

# Populate the plot with our line coming from the Data Source
x_allele_ax.line(x='time', y='x_allele', line_width=2, source=source)

GlyphRenderer(id='2670', ...)

Now let'set the format for the plotbb

In [44]:
theme_json = {'attrs':
            {'Figure': {
                'background_fill_color': '#E3DCD0',
                'outline_line_color': '#FFFFFF',
            },
            'Axis': {
            'axis_line_color': "white",
            'major_tick_in': 7,
            'major_tick_line_width': 2.5,
            'major_tick_line_color': "white",
            'minor_tick_line_color': "white",
            'axis_label_text_font': 'Helvetica',
            'axis_label_text_font_style': 'normal'
            },
            'Grid': {
                'grid_line_color': None,
            },
            'Legend': {
                'background_fill_color': '#E3DCD0',
                'border_line_color': '#FFFFFF',
                'border_line_width': 1.5,
                'background_fill_alpha': 0.5
            },
            'Text': {
                'text_font_style': 'normal',
               'text_font': 'Helvetica'
            },
            'Title': {
                'background_fill_color': '#FFEDC0',
                'text_font_style': 'normal',
                'align': 'center',
                'text_font': 'Helvetica',
                'offset': 2,
            }}}

theme = Theme(json=theme_json)
bokeh.io.curdoc().theme = theme

In [45]:
fig = bokeh.layouts.column(x_init, s_select, x_allele_ax)
bokeh.io.show(fig)